# Import Libraries

In [2]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from operator import add
from pyspark import SparkContext
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Data Preprocessing

In [3]:
df = pd.read_csv("data\Combined_data.csv")
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [4]:
def Func_1(line):
    strs = []   
    line = line.replace("\"", "")
    line = line.split(",b")
    for idx,string in enumerate(line):
        if(idx>=1):
            strs.append(string)
    return strs

In [5]:
sc =SparkContext()
rdd = sc.textFile('data/new_data.csv')
lines = rdd.map(Func_1)
docs=lines.take(1989)
sc.stop()

In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
scores = []
for idx,line in enumerate(docs):

    neg = 0
    neu = 0
    pos = 0
    compound = 0
    for sentence in line:
        cur_scores = sid.polarity_scores(sentence)
        neg += cur_scores['neg']
        neu += cur_scores['neu']
        pos += cur_scores['pos']
        compound += cur_scores['compound']
    
    scores.append([round(neg/25,3),round(neu/25,3),round(pos/25,3),round(compound/25,3)])


In [10]:
scores = np.array(scores)
neg_score = scores[:,0]
neu_score = scores[:,1]
pos_score = scores[:,2]
compound_score = scores[:,3]

#### Merge two datasets

In [11]:
df_basic = pd.read_csv("data\Combined_data.csv")
df_stock = pd.read_csv("data\Stock_data.csv")
merged_df = df_basic[['Date', 'Label']].merge(df_stock, how='inner', on='Date', right_index=True)
merged_df['Neg'] = neg_score
merged_df['Neu'] = neu_score
merged_df['Pos'] = pos_score
merged_df['Compound'] = compound_score
merged_df.head()

,Date,Label,Open,High,Low,Close,Volume,Adj Close,Neg,Neu,Pos,Compound
0,2008-08-08,0,11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312,0.203,0.761,0.036,-0.320
1,2008-08-11,1,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609,0.181,0.743,0.076,-0.154
2,2008-08-12,0,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727,0.175,0.767,0.058,-0.242
3,2008-08-13,0,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961,0.149,0.793,0.058,-0.142
4,2008-08-14,1,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688,0.175,0.742,0.083,-0.179


In [238]:
merged_df.to_csv("data/marged_data.csv",index=False,header=1)

In [12]:
y_all = np.array(merged_df[['Label']]).ravel()
x_all = np.array(merged_df[['Open','High','Low','Close','Volume','Adj Close','Neg','Neu','Pos','Compound']])

In [13]:
x_train,x_test,y_train,y_test = train_test_split(x_all,y_all,test_size=0.3,random_state=0)


#### Try different models

In [14]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier


In [15]:
models = []
models.append(('LDA' , LinearDiscriminantAnalysis()))
models.append(('KNN' , KNeighborsClassifier()))
models.append(('NB' , GaussianNB()))
models.append(('SVM' , SVC()))
models.append(('RF' , RandomForestClassifier(n_estimators=50)))


In [16]:

for name, model in models:
    clf = model
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    accu_score = accuracy_score(y_test, y_pred)
    print(name + ": ")
    print(accu_score)

C:\Users\NingyuanZhang\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LDA: 
0.9179229480737019
KNN: 
0.5025125628140703
NB: 
0.5376884422110553
SVM: 
0.5293132328308208
RF: 
0.7470686767169179
